In [1]:
import qiskit
from qiskit import transpile
import numpy as np
import pandas as pd
import yfinance as yf
import sys
import os
os.environ["Q_ALCHEMY_API_KEY"] = "JnvkpMCsyr4nB9nHcwa6CbxqhtZXyF1b"
sys.path.append('..')
from q_alchemy.qiskit import QAlchemyInitialize
 

In [3]:
#tickers = ['TSLA', 'GOOGL', 'AMZN', 'AAPL', 'META']
tickers = ['TSLA', 'GOOGL']
start_date = '2021-01-01'
end_date = '2022-01-01'

data = {}

for ticker in tickers:
    data[ticker] =  yf.download(ticker, start=start_date, end=end_date)

print(data)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
{'TSLA':                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-01-04  239.820007  248.163330  239.063339  243.256668  243.256668   
2021-01-05  241.220001  246.946671  239.733337  245.036667  245.036667   
2021-01-06  252.830002  258.000000  249.699997  251.993332  251.993332   
2021-01-07  259.209991  272.329987  258.399994  272.013336  272.013336   
2021-01-08  285.333344  294.829987  279.463318  293.339996  293.339996   
...                ...         ...         ...         ...         ...   
2021-12-27  357.890015  372.333344  356.906677  364.646667  364.646667   
2021-12-28  369.829987  373.000000  359.473328  362.823334  362.823334   
2021-12-29  366.213318  368.000000  354.713318  362.063324  362.063324   
2021-12-30  353.776672  365.183319  351

In [4]:
for ticker in tickers:
    data[ticker]['Daily Return'] =  data[ticker]['Adj Close'].pct_change()
    data[ticker] = data[ticker].dropna()

In [5]:
expected_returns = {}
std_deviation = {}

for ticker in tickers:
    expected_returns[ticker] = data[ticker]['Daily Return'].mean()
    std_deviation[ticker] = data[ticker]['Daily Return'].std()
    
print(expected_returns)
print(std_deviation)

{'TSLA': 0.002061082797253658, 'GOOGL': 0.0021812108248734545}
{'TSLA': 0.03450369927285423, 'GOOGL': 0.015311596310057506}


In [6]:
y_raw = [list(data[ticker]['Daily Return'].values) for ticker in tickers]
y_new = list(map(list, zip(*y_raw)))
y_new = np.array(y_new)

t = len(y_new)
n = len(y_new[0])

R = [expected_returns[ticker] for ticker in tickers]
R = np.array(R)

cov_matrix = np.zeros((n, n))

for i in range(t):
    cov_matrix += np.outer((y_new[i]-R), (y_new[i]-R))
cov_matrix = cov_matrix/(t-1)
print(cov_matrix)

[[0.00119051 0.00015751]
 [0.00015751 0.00023444]]


In [7]:
from datetime import datetime

yesterday = '2023-12-20'
today = '2023-12-21'

today_price = []

for ticker in tickers:
    data = yf.download(ticker, start=yesterday, end=today)
    today_price.append(data['Open'].values[0])

print(today_price)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[256.4100036621094, 138.97000122070312]


In [12]:
A = np.zeros((2*n, 2*n))
for i in range(2):
    A[0][i+2] = R[i]
    A[1][i+2] = today_price[i]
    A[i+2][0] = R[i]
    A[i+2][1] = today_price[i]
A[2:, 2:4] =   cov_matrix
print(A)

[[0.00000000e+00 0.00000000e+00 2.06108280e-03 2.18121082e-03]
 [0.00000000e+00 0.00000000e+00 2.56410004e+02 1.38970001e+02]
 [2.06108280e-03 2.56410004e+02 1.19050526e-03 1.57506067e-04]
 [2.18121082e-03 1.38970001e+02 1.57506067e-04 2.34444982e-04]]


In [ ]:
mu = np.dot(R, )


[[0.00000000e+00 0.00000000e+00 2.06108280e-03 2.18121082e-03]
 [0.00000000e+00 0.00000000e+00 2.56410004e+02 1.38970001e+02]
 [2.06108280e-03 2.56410004e+02 0.00000000e+00 0.00000000e+00]
 [2.18121082e-03 1.38970001e+02 0.00000000e+00 0.00000000e+00]]
